<a href="https://colab.research.google.com/github/ahmedabdulghany7/Competitive-Intelligence-Multi-Agent-System/blob/main/Action_Recommender_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Action Recommender Agent (LLM-Powered)
Competitive Intelligence Multi-Agent System

Purpose: Generate specific, actionable recommendations to answer "What should we do?"
Target: Suggest actionable procedures
Functions:
- Identify appropriate responses
- Prioritize actions
- Suggest implementation timing
Output: Priority-ordered action list
"""

'\nAction Recommender Agent (LLM-Powered)\nCompetitive Intelligence Multi-Agent System\n\nPurpose: Generate specific, actionable recommendations to answer "What should we do?"\nTarget: Suggest actionable procedures\nFunctions:\n- Identify appropriate responses\n- Prioritize actions\n- Suggest implementation timing\nOutput: Priority-ordered action list\n'

In [ ]:


import json
import datetime
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
from enum import Enum
import openai
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ActionPriority(Enum):
    """Action priority levels"""
    CRITICAL = "Critical"
    HIGH = "High"
    MEDIUM = "Medium"
    LOW = "Low"

class ActionCategory(Enum):
    """Categories of recommended actions"""
    IMMEDIATE = "Immediate Response"
    PRODUCT = "Product Strategy"
    MARKETING = "Marketing & Communication"
    PRICING = "Pricing Strategy"
    PARTNERSHIP = "Partnerships & Alliances"
    RESEARCH = "Research & Development"
    MARKET_EXPANSION = "Market Expansion"
    COMPETITIVE_DEFENSE = "Competitive Defense"
    RETAIL = "Retail & Channel"          # <-- added
    SOFTWARE = "Software & Features"
    HARDWARE = "Hardware & Devices"

@dataclass
class ActionRecommendation:
    """Structure for action recommendations"""
    action_id: str
    title: str
    description: str
    category: ActionCategory
    priority: ActionPriority
    urgency_hours: int  # How quickly to implement (in hours)
    estimated_effort: str  # Low, Medium, High
    estimated_cost: str  # Low, Medium, High
    expected_impact: str  # Low, Medium, High
    success_metrics: List[str]
    implementation_steps: List[str]
    risks: List[str]
    dependencies: List[str]
    stakeholders: List[str]
    confidence_score: float  # 0.0 to 1.0
    rationale: str
    created_at: datetime.datetime

class ActionRecommenderAgent:
    """
    LLM-powered agent that generates actionable recommendations
    based on competitive intelligence analysis
    """

    def __init__(self, openai_api_key: str = None, model: str = "gpt-4"):
        """
        Initialize the Action Recommender Agent

        Args:
            openai_api_key: OpenAI API key
            model: LLM model to use (default: gpt-4)
        """
        self.model = model
        self.client = openai.OpenAI(api_key=openai_api_key) if openai_api_key else None
        self.action_history = []

        # Action templates and frameworks
        self.action_frameworks = {
            "product_launch": self._get_product_launch_framework(),
            "pricing_change": self._get_mobile_pricing_framework(),   # <-- كان _get_pricing_framework()
            "marketing_campaign": self._get_marketing_framework(),    # أو: self._get_mobile_marketing_framework()
            "expansion": self._get_expansion_framework(),             # أو: self._get_mobile_expansion_framework()
            "leadership_change": self._get_leadership_framework(),
            "funding_investment": self._get_funding_framework()
        }

    def analyze_and_recommend(self,
                            competitive_event: Dict,
                            impact_score: float,
                            strategic_context: str,
                            company_profile: Dict) -> List[ActionRecommendation]:
        """
        Main method to analyze competitive intelligence and generate recommendations

        Args:
            competitive_event: Details of the competitive event
            impact_score: Impact score (0-10) from Impact Scoring Agent
            strategic_context: Strategic analysis from Strategic Analyst Agent
            company_profile: Our company's profile and capabilities

        Returns:
            List of ActionRecommendation objects
        """
        try:
            # Determine urgency based on impact score
            urgency_level = self._determine_urgency(impact_score)

            # Generate recommendations using LLM
            if self.client:
                recommendations = self._generate_llm_recommendations(
                    competitive_event, impact_score, strategic_context, company_profile
                )
            else:
                # Fallback to rule-based recommendations
                recommendations = self._generate_rule_based_recommendations(
                    competitive_event, impact_score, strategic_context, company_profile
                )

            # Post-process and validate recommendations
            validated_recommendations = self._validate_and_rank_recommendations(
                recommendations, urgency_level
            )

            # Store in history
            self.action_history.extend(validated_recommendations)

            logger.info(f"Generated {len(validated_recommendations)} action recommendations")
            return validated_recommendations

        except Exception as e:
            logger.error(f"Error in analyze_and_recommend: {str(e)}")
            return []

    def _determine_urgency(self, impact_score: float) -> str:
        """Determine urgency level based on impact score"""
        if impact_score >= 8.5:
            return "immediate"  # Within 24-48 hours
        elif impact_score >= 7.0:
            return "high"      # Within 1 week
        elif impact_score >= 5.0:
            return "medium"    # Within 1 month
        else:
            return "low"       # Within 3 months

    def _generate_llm_recommendations(self,
                                    competitive_event: Dict,
                                    impact_score: float,
                                    strategic_context: str,
                                    company_profile: Dict) -> List[Dict]:
        """Generate recommendations using LLM"""

        # Construct detailed prompt
        prompt = self._build_recommendation_prompt(
            competitive_event, impact_score, strategic_context, company_profile
        )

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self._get_system_prompt()},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2000
            )

            # Parse LLM response
            recommendations_text = response.choices[0].message.content
            recommendations = self._parse_llm_response(recommendations_text)

            return recommendations

        except Exception as e:
            logger.error(f"LLM recommendation generation failed: {str(e)}")
            return []

    def _generate_rule_based_recommendations(self,
                                           competitive_event: Dict,
                                           impact_score: float,
                                           strategic_context: str,
                                           company_profile: Dict) -> List[Dict]:
        """Fallback rule-based recommendation generation"""

        event_type = competitive_event.get('event_type', 'unknown')
        competitor_name = competitive_event.get('competitor', 'Unknown Competitor')

        recommendations = []

        # Get framework for event type
        framework = self.action_frameworks.get(event_type, self.action_frameworks['product_launch'])

        # Generate recommendations based on impact score and event type
        if impact_score >= 8.5:  # Critical
            recommendations.extend(framework['critical'])
        elif impact_score >= 7.0:  # High
            recommendations.extend(framework['high'])
        elif impact_score >= 5.0:  # Medium
            recommendations.extend(framework['medium'])
        else:  # Low
            recommendations.extend(framework['low'])

        # Customize recommendations
        for rec in recommendations:
            rec['competitor'] = competitor_name
            rec['event_details'] = competitive_event.get('description', '')

        return recommendations

    def _validate_and_rank_recommendations(self,
                                         raw_recommendations: List[Dict],
                                         urgency_level: str) -> List[ActionRecommendation]:
        """Validate and rank recommendations by priority and feasibility"""

        validated_recommendations = []

        for i, rec in enumerate(raw_recommendations):
            try:
                # Create ActionRecommendation object
                action_rec = ActionRecommendation(
                    action_id=f"ACT_{datetime.datetime.now().strftime('%Y%m%d')}_{i+1:03d}",
                    title=rec.get('title', 'Untitled Action'),
                    description=rec.get('description', ''),
                    category=ActionCategory(rec.get('category', ActionCategory.IMMEDIATE.value)),
                    priority=ActionPriority(rec.get('priority', ActionPriority.MEDIUM.value)),
                    urgency_hours=rec.get('urgency_hours', 168),  # Default 1 week
                    estimated_effort=rec.get('estimated_effort', 'Medium'),
                    estimated_cost=rec.get('estimated_cost', 'Medium'),
                    expected_impact=rec.get('expected_impact', 'Medium'),
                    success_metrics=rec.get('success_metrics', []),
                    implementation_steps=rec.get('implementation_steps', []),
                    risks=rec.get('risks', []),
                    dependencies=rec.get('dependencies', []),
                    stakeholders=rec.get('stakeholders', []),
                    confidence_score=rec.get('confidence_score', 0.7),
                    rationale=rec.get('rationale', ''),
                    created_at=datetime.datetime.now()
                )

                validated_recommendations.append(action_rec)

            except Exception as e:
                logger.warning(f"Failed to validate recommendation {i}: {str(e)}")
                continue

        # Sort by priority and urgency
        priority_order = {
            ActionPriority.CRITICAL: 4,
            ActionPriority.HIGH: 3,
            ActionPriority.MEDIUM: 2,
            ActionPriority.LOW: 1
        }

        validated_recommendations.sort(
            key=lambda x: (priority_order[x.priority], -x.urgency_hours, -x.confidence_score),
            reverse=True
        )

        return validated_recommendations

    def _build_recommendation_prompt(self,
                                   competitive_event: Dict,
                                   impact_score: float,
                                   strategic_context: str,
                                   company_profile: Dict) -> str:
        """Build detailed prompt for LLM"""

        prompt = f"""
COMPETITIVE INTELLIGENCE ANALYSIS REQUEST

EVENT DETAILS:
- Event Type: {competitive_event.get('event_type', 'Unknown')}
- Competitor: {competitive_event.get('competitor', 'Unknown')}
- Description: {competitive_event.get('description', 'No description')}
- Impact Score: {impact_score}/10
- Event Date: {competitive_event.get('date', 'Unknown')}

STRATEGIC CONTEXT:
{strategic_context}

OUR COMPANY PROFILE:
- Company Size: {company_profile.get('size', 'Medium')}
- Market Position: {company_profile.get('market_position', 'Competitive')}
- Core Strengths: {', '.join(company_profile.get('strengths', []))}
- Available Resources: {company_profile.get('resources', 'Standard')}
- Key Markets: {', '.join(company_profile.get('markets', []))}

REQUIREMENTS:
Please provide 3-5 specific, actionable recommendations that answer "What should we do?"

For each recommendation, include:
1. Title (brief, actionable)
2. Description (detailed explanation)
3. Category (Immediate Response, Product Strategy, Marketing & Communication, etc.)
4. Priority (Critical, High, Medium, Low)
5. Urgency in hours
6. Estimated effort (Low, Medium, High)
7. Estimated cost (Low, Medium, High)
8. Expected impact (Low, Medium, High)
9. Success metrics (3-5 measurable outcomes)
10. Implementation steps (detailed action plan)
11. Risks (potential challenges)
12. Dependencies (what needs to happen first)
13. Stakeholders (who needs to be involved)
14. Confidence score (0.0-1.0)
15. Rationale (why this action is recommended)

Focus on recommendations that are:
- Specific and actionable
- Appropriate for our company size and resources
- Responsive to the competitive threat/opportunity
- Measurable and time-bound
"""
        return prompt

    def _get_system_prompt(self) -> str:
        """Get system prompt for LLM"""
        return """
You are a senior strategic analyst specializing in competitive intelligence and business strategy.
Your role is to analyze competitive events and provide specific, actionable recommendations.

Key principles:
1. Be specific and actionable - avoid generic advice
2. Consider resource constraints and company capabilities
3. Prioritize based on impact and urgency
4. Provide measurable success criteria
5. Include implementation details and timelines
6. Identify potential risks and mitigation strategies
7. Be realistic about what can be achieved

Format your response as a structured list with clear sections for each recommendation.
"""

    def _parse_llm_response(self, response_text: str) -> List[Dict]:
        """Parse LLM response into structured recommendations"""
        # This is a simplified parser - in production, you'd want more sophisticated parsing
        recommendations = []

        # Split response into sections (basic implementation)
        lines = response_text.split('\n')
        current_rec = {}

        for line in lines:
            line = line.strip()
            if line.startswith('**') or line.startswith('##'):
                # New recommendation section
                if current_rec:
                    recommendations.append(current_rec)
                    current_rec = {}
            elif ':' in line:
                # Key-value pair
                key, value = line.split(':', 1)
                key = key.strip().lower().replace(' ', '_')
                value = value.strip()
                current_rec[key] = value

        if current_rec:
            recommendations.append(current_rec)

        return recommendations

    def _get_product_launch_framework(self) -> Dict:
        """Framework for responding to product launches"""
        return {
            'critical': [
                {
                    'title': 'Emergency Competitive Analysis',
                    'description': 'Conduct immediate deep-dive analysis of competitor product',
                    'category': ActionCategory.IMMEDIATE.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 24,
                    'estimated_effort': 'High',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'High'
                },
                {
                    'title': 'Accelerate Product Roadmap',
                    'description': 'Fast-track competing features or products',
                    'category': ActionCategory.PRODUCT.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 72
                }
            ],
            'high': [
                {
                    'title': 'Feature Gap Analysis',
                    'description': 'Analyze feature gaps and enhancement opportunities',
                    'category': ActionCategory.PRODUCT.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 168
                },
                {
                    'title': 'Marketing Counter-Campaign',
                    'description': 'Launch marketing campaign highlighting our advantages',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 120
                }
            ],
            'medium': [
                {
                    'title': 'Market Research Study',
                    'description': 'Conduct customer perception study',
                    'category': ActionCategory.RESEARCH.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 720
                }
            ],
            'low': [
                {
                    'title': 'Monitor Market Response',
                    'description': 'Track competitor product performance',
                    'category': ActionCategory.IMMEDIATE.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 2160
                }
            ]
        }

    def _get_mobile_pricing_framework(self) -> Dict:
        """Framework for responding to mobile pricing changes"""
        return {
            'critical': [
                {
                    'title': 'Mobile Pricing Strategy Emergency Review',
                    'description': 'Immediate review of pricing strategy across all phone models',
                    'category': ActionCategory.PRICING.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 48,
                    'estimated_effort': 'High',
                    'estimated_cost': 'Low',
                    'expected_impact': 'High',
                    'success_metrics': ['Price competitiveness analysis', 'Margin impact assessment'],
                    'implementation_steps': [
                        'Compare pricing across all models and variants',
                        'Analyze value proposition at each price point',
                        'Assess margin impact of potential price changes',
                        'Review carrier partnership pricing',
                        'Evaluate promotional pricing strategies'
                    ],
                    'risks': ['Price war escalation', 'Margin compression'],
                    'dependencies': ['Sales and finance data'],
                    'stakeholders': ['Pricing Team', 'Finance', 'Sales'],
                    'confidence_score': 0.9,
                    'rationale': 'Pricing changes can immediately impact sales and market share'
                }
            ],
            'high': [
                {
                    'title': 'Value Proposition Enhancement',
                    'description': 'Strengthen phone value proposition to justify current pricing',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 168,
                    'estimated_effort': 'Medium',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'Medium',
                    'success_metrics': ['Improved price perception', 'Better feature-to-price ratio communication'],
                    'implementation_steps': [
                        'Highlight unique features and advantages',
                        'Create comparison charts vs competitors',
                        'Emphasize build quality and reliability',
                        'Showcase camera capabilities and performance',
                        'Promote ecosystem integration benefits'
                    ],
                    'risks': ['Overclaiming features', 'Competitor response'],
                    'dependencies': ['Marketing creative development'],
                    'stakeholders': ['Marketing', 'Product Marketing', 'PR'],
                    'confidence_score': 0.8,
                    'rationale': 'Strong value communication can offset pricing pressure'
                }
            ],
            'medium': [
                {
                    'title': 'Mobile Customer Retention Program',
                    'description': 'Launch loyalty program to retain price-sensitive customers',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 336,
                    'estimated_effort': 'Medium',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'Medium',
                    'success_metrics': ['Customer churn reduction', 'Repeat purchase rate'],
                    'implementation_steps': [
                        'Design loyalty rewards program',
                        'Offer trade-in incentives',
                        'Create upgrade programs',
                        'Partner with service providers',
                        'Implement referral bonuses'
                    ],
                    'risks': ['Program costs', 'Low adoption'],
                    'dependencies': ['CRM system capabilities'],
                    'stakeholders': ['Customer Service', 'Marketing', 'Retail'],
                    'confidence_score': 0.7,
                    'rationale': 'Retention is more cost-effective than acquisition'
                }
            ],
            'low': [
                {
                    'title': 'Long-term Mobile Pricing Strategy',
                    'description': 'Develop comprehensive long-term pricing strategy',
                    'category': ActionCategory.PRICING.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 1440,
                    'estimated_effort': 'High',
                    'estimated_cost': 'Low',
                    'expected_impact': 'High',
                    'success_metrics': ['Strategic pricing model', 'Market positioning clarity'],
                    'implementation_steps': [
                        'Analyze global pricing trends',
                        'Develop segment-specific pricing',
                        'Create dynamic pricing capabilities',
                        'Plan lifecycle pricing strategies',
                        'Build competitive pricing intelligence'
                    ],
                    'risks': ['Market complexity', 'Execution challenges'],
                    'dependencies': ['Market research data'],
                    'stakeholders': ['Strategy', 'Pricing', 'Finance'],
                    'confidence_score': 0.8,
                    'rationale': 'Long-term pricing strategy provides sustainable competitive advantage'
                }
            ]
        }

    def _get_mobile_marketing_framework(self) -> Dict:
        """Framework for responding to mobile marketing campaigns"""
        return {
            'critical': [
                {
                    'title': 'Mobile Counter-Marketing Campaign',
                    'description': 'Launch immediate counter-marketing for mobile phones',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 72,
                    'estimated_effort': 'High',
                    'estimated_cost': 'High',
                    'expected_impact': 'High',
                    'success_metrics': ['Brand awareness maintenance', 'Purchase consideration'],
                    'implementation_steps': [
                        'Create comparison campaigns highlighting our advantages',
                        'Launch social media response campaign',
                        'Partner with tech influencers and reviewers',
                        'Organize hands-on demo events',
                        'Run targeted digital advertising'
                    ],
                    'risks': ['Negative brand association', 'Legal issues'],
                    'dependencies': ['Marketing budget', 'Creative assets'],
                    'stakeholders': ['Marketing', 'PR', 'Legal', 'Influencer Relations'],
                    'confidence_score': 0.8,
                    'rationale': 'Quick response needed to counter competitor messaging'
                }
            ]
        }

    def _get_mobile_expansion_framework(self) -> Dict:
        """Framework for responding to mobile market expansion"""
        return {
            'critical': [
                {
                    'title': 'Mobile Market Entry Assessment',
                    'description': 'Assess our entry options in competitor new market',
                    'category': ActionCategory.MARKET_EXPANSION.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 168,
                    'estimated_effort': 'High',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'High',
                    'success_metrics': ['Market opportunity assessment', 'Entry strategy development'],
                    'implementation_steps': [
                        'Analyze market size and growth potential',
                        'Assess local competition and regulations',
                        'Evaluate distribution and retail partners',
                        'Study consumer preferences and price sensitivity',
                        'Plan localization requirements'
                    ],
                    'risks': ['Regulatory barriers', 'Local competition'],
                    'dependencies': ['Market research capabilities'],
                    'stakeholders': ['Strategy', 'International Business', 'Regulatory'],
                    'confidence_score': 0.7,
                    'rationale': 'New market entry requires careful assessment of opportunities'
                }
            ]
        }

    def _get_mobile_feature_framework(self) -> Dict:
        """Framework for responding to new mobile features"""
        return {
            'critical': [
                {
                    'title': 'Feature Parity Development',
                    'description': 'Develop competing features for our mobile phones',
                    'category': ActionCategory.SOFTWARE.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 336,
                    'estimated_effort': 'High',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'High',
                    'success_metrics': ['Feature implementation', 'User adoption rate'],
                    'implementation_steps': [
                        'Analyze competitor feature implementation',
                        'Design our version with improvements',
                        'Develop and test the feature',
                        'Plan software update rollout',
                        'Create feature marketing campaign'
                    ],
                    'risks': ['Development delays', 'Patent issues'],
                    'dependencies': ['Engineering resources', 'Legal clearance'],
                    'stakeholders': ['Software Engineering', 'Product Management', 'Legal'],
                    'confidence_score': 0.7,
                    'rationale': 'Feature gaps can impact competitive positioning'
                }
            ]
        }

    def _get_hardware_framework(self) -> Dict:
        """Framework for responding to hardware upgrades"""
        return {
            'critical': [
                {
                    'title': 'Hardware Roadmap Acceleration',
                    'description': 'Accelerate hardware development to match competition',
                    'category': ActionCategory.HARDWARE.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 720,
                    'estimated_effort': 'High',
                    'estimated_cost': 'High',
                    'expected_impact': 'High',
                    'success_metrics': ['Hardware spec improvements', 'Time-to-market reduction'],
                    'implementation_steps': [
                        'Review current hardware roadmap',
                        'Identify key specifications to improve',
                        'Source better components and suppliers',
                        'Adjust manufacturing processes',
                        'Plan next generation phone features'
                    ],
                    'risks': ['Supply chain disruption', 'Quality issues'],
                    'dependencies': ['Supplier partnerships', 'Manufacturing capacity'],
                    'stakeholders': ['Hardware Engineering', 'Supply Chain', 'Manufacturing'],
                    'confidence_score': 0.6,
                    'rationale': 'Hardware upgrades require longer lead times but are critical'
                }
            ]
        }

    def _get_mobile_partnership_framework(self) -> Dict:
        """Framework for mobile partnership responses"""
        return {
            'high': [
                {
                    'title': 'Strategic Mobile Partnerships',
                    'description': 'Forge partnerships to strengthen mobile ecosystem',
                    'category': ActionCategory.PARTNERSHIP.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 336,
                    'estimated_effort': 'Medium',
                    'estimated_cost': 'Medium',
                    'expected_impact': 'High',
                    'success_metrics': ['Partnership agreements signed', 'Ecosystem integration'],
                    'implementation_steps': [
                        'Identify potential carrier partners',
                        'Negotiate exclusive app partnerships',
                        'Partner with accessory manufacturers',
                        'Collaborate with content providers',
                        'Build developer ecosystem'
                    ],
                    'risks': ['Partner conflicts', 'Exclusive dependencies'],
                    'dependencies': ['Business development resources'],
                    'stakeholders': ['Business Development', 'Partnerships', 'Legal'],
                    'confidence_score': 0.8,
                    'rationale': 'Strong partnerships enhance competitive position'
                }
            ]
        }

    def _get_retail_framework(self) -> Dict:
        """Framework for retail expansion responses"""
        return {
            'high': [
                {
                    'title': 'Mobile Retail Channel Expansion',
                    'description': 'Expand retail presence to match competitor reach',
                    'category': ActionCategory.RETAIL.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 720,
                    'estimated_effort': 'High',
                    'estimated_cost': 'High',
                    'expected_impact': 'Medium',
                    'success_metrics': ['Retail points increase', 'Market coverage expansion'],
                    'implementation_steps': [
                        'Identify key retail locations and partners',
                        'Negotiate retail partnerships',
                        'Set up branded retail displays',
                        'Train retail staff on product features',
                        'Launch retail-specific promotions'
                    ],
                    'risks': ['High retail costs', 'Channel conflicts'],
                    'dependencies': ['Retail partnership agreements'],
                    'stakeholders': ['Retail', 'Sales', 'Marketing'],
                    'confidence_score': 0.7,
                    'rationale': 'Retail presence directly impacts sales accessibility'
                },
                {
                    'title': 'Value Proposition Enhancement',
                    'description': 'Strengthen value proposition to justify pricing',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 168
                }
            ],
            'medium': [
                {
                    'title': 'Customer Retention Campaign',
                    'description': 'Launch campaign to retain price-sensitive customers',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 336
                }
            ],
            'low': [
                {
                    'title': 'Long-term Pricing Strategy',
                    'description': 'Develop long-term pricing strategy',
                    'category': ActionCategory.PRICING.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 1440
                }
            ]
        }


    def _get_marketing_framework(self) -> Dict:
        """Framework for responding to marketing campaigns"""
        return {
            'critical': [
                {
                    'title': 'Counter-Marketing Campaign',
                    'description': 'Launch immediate counter-marketing initiative',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 72
                }
            ],
            'high': [
                {
                    'title': 'Message Differentiation',
                    'description': 'Develop differentiated marketing messages',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 120
                }
            ],
            'medium': [
                {
                    'title': 'Brand Awareness Campaign',
                    'description': 'Increase brand awareness in targeted segments',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 504
                }
            ],
            'low': [
                {
                    'title': 'Marketing Performance Analysis',
                    'description': 'Analyze effectiveness of competitor campaign',
                    'category': ActionCategory.MARKETING.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 1680
                }
            ]
        }

    def _get_expansion_framework(self) -> Dict:
        """Framework for responding to market expansion"""
        return {
            'critical': [
                {
                    'title': 'Market Entry Assessment',
                    'description': 'Assess our entry options in the same market',
                    'category': ActionCategory.MARKET_EXPANSION.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 168
                }
            ],
            'high': [
                {
                    'title': 'Partnership Opportunities',
                    'description': 'Explore strategic partnerships in target market',
                    'category': ActionCategory.PARTNERSHIP.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 336
                }
            ],
            'medium': [
                {
                    'title': 'Market Research',
                    'description': 'Conduct detailed market research',
                    'category': ActionCategory.RESEARCH.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 720
                }
            ],
            'low': [
                {
                    'title': 'Long-term Expansion Planning',
                    'description': 'Develop long-term expansion strategy',
                    'category': ActionCategory.MARKET_EXPANSION.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 2160
                }
            ]
        }

    def _get_leadership_framework(self) -> Dict:
        """Framework for responding to leadership changes"""
        return {
            'critical': [
                {
                    'title': 'Leadership Analysis',
                    'description': 'Analyze new leader background and strategic implications',
                    'category': ActionCategory.IMMEDIATE.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 72
                }
            ],
            'high': [
                {
                    'title': 'Relationship Building',
                    'description': 'Identify opportunities for relationship building',
                    'category': ActionCategory.PARTNERSHIP.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 336
                }
            ],
            'medium': [
                {
                    'title': 'Strategic Direction Monitoring',
                    'description': 'Monitor changes in competitor strategy',
                    'category': ActionCategory.IMMEDIATE.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 720
                }
            ],
            'low': [
                {
                    'title': 'Industry Network Strengthening',
                    'description': 'Strengthen industry relationships and networks',
                    'category': ActionCategory.PARTNERSHIP.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 1440
                }
            ]
        }

    def _get_funding_framework(self) -> Dict:
        """Framework for responding to funding/investment news"""
        return {
            'critical': [
                {
                    'title': 'Competitive Threat Assessment',
                    'description': 'Assess increased competitive threat from funding',
                    'category': ActionCategory.IMMEDIATE.value,
                    'priority': ActionPriority.CRITICAL.value,
                    'urgency_hours': 48
                }
            ],
            'high': [
                {
                    'title': 'Strategic Partnership Acceleration',
                    'description': 'Accelerate our own partnership or funding initiatives',
                    'category': ActionCategory.PARTNERSHIP.value,
                    'priority': ActionPriority.HIGH.value,
                    'urgency_hours': 168
                }
            ],
            'medium': [
                {
                    'title': 'Innovation Investment',
                    'description': 'Increase investment in innovation and R&D',
                    'category': ActionCategory.RESEARCH.value,
                    'priority': ActionPriority.MEDIUM.value,
                    'urgency_hours': 720
                }
            ],
            'low': [
                {
                    'title': 'Market Position Reinforcement',
                    'description': 'Reinforce market position and competitive advantages',
                    'category': ActionCategory.COMPETITIVE_DEFENSE.value,
                    'priority': ActionPriority.LOW.value,
                    'urgency_hours': 1440
                }
            ]
        }

    def generate_action_report(self, recommendations: List[ActionRecommendation]) -> str:
        """Generate formatted action report"""

        if not recommendations:
            return "No action recommendations generated."

        report = f"""
# ACTION RECOMMENDATIONS REPORT
Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## EXECUTIVE SUMMARY
Total Recommendations: {len(recommendations)}
Critical Actions: {len([r for r in recommendations if r.priority == ActionPriority.CRITICAL])}
High Priority Actions: {len([r for r in recommendations if r.priority == ActionPriority.HIGH])}

## RECOMMENDED ACTIONS

"""

        for i, rec in enumerate(recommendations, 1):
            report += f"""
### {i}. {rec.title}
**Priority:** {rec.priority.value} | **Category:** {rec.category.value}
**Urgency:** {rec.urgency_hours} hours | **Effort:** {rec.estimated_effort}
**Expected Impact:** {rec.expected_impact} | **Confidence:** {rec.confidence_score:.2f}

**Description:** {rec.description}

**Implementation Steps:**
"""
            for step in rec.implementation_steps:
                report += f"- {step}\n"

            report += f"""
**Success Metrics:**
"""
            for metric in rec.success_metrics:
                report += f"- {metric}\n"

            report += f"""
**Key Risks:**
"""
            for risk in rec.risks:
                report += f"- {risk}\n"

            report += f"""
**Stakeholders:** {', '.join(rec.stakeholders)}
**Dependencies:** {', '.join(rec.dependencies)}

**Rationale:** {rec.rationale}

---
"""

        return report

    def get_immediate_actions(self, recommendations: List[ActionRecommendation]) -> List[ActionRecommendation]:
        """Filter and return only immediate actions (urgent within 48 hours)"""
        return [rec for rec in recommendations if rec.urgency_hours <= 48]

    def get_actions_by_priority(self,
                              recommendations: List[ActionRecommendation],
                              priority: ActionPriority) -> List[ActionRecommendation]:
        """Filter recommendations by priority level"""
        return [rec for rec in recommendations if rec.priority == priority]

# Example usage and testing for Mobile Phone Market
if __name__ == "__main__":
    print("📱 Action Recommender Agent - Mobile Phone Competitive Intelligence")
    print("=" * 65)

    # Initialize agent
    agent = ActionRecommenderAgent()

    # Sample mobile competitive events
    mobile_events = [
        {
            'event_type': 'product_launch',
            'competitor': 'Samsung',
            'description': 'Samsung Galaxy S25 Ultra launched with 200MP camera and 5000mAh battery',
            'date': '2024-01-15',
            'source': 'Samsung Official',
            'product_category': 'Premium Smartphone',
            'target_segment': 'Premium',
            'key_features': ['200MP Camera', '5000mAh Battery', '12GB RAM', '1TB Storage'],
            'price_range': '$1200-1400'
        },
        {
            'event_type': 'pricing_change',
            'competitor': 'Oppo',
            'description': 'Oppo reduces Find X7 Pro price by 20% in Middle East markets',
            'date': '2024-01-10',
            'source': 'Oppo Middle East',
            'product_category': 'Mid-Premium Smartphone',
            'target_segment': 'Mid-Premium',
            'price_range': '$600-800'
        },
        {
            'event_type': 'marketing_campaign',
            'competitor': 'Apple',
            'description': 'Apple launches "Shot on iPhone" campaign focusing on mobile photography',
            'date': '2024-01-12',
            'source': 'Apple Marketing',
            'target_segment': 'Creative Professionals'
        }
    ]

    # Sample mobile company profile
    mobile_company_profile = {
        'size': 'Medium',
        'market_position': 'Strong in Mid-Range',
        'strengths': ['Fast Charging', 'Build Quality', 'Customer Service', 'Value for Money'],
        'resources': 'Medium',
        'markets': ['Middle East', 'North Africa', 'Southeast Asia'],
        'product_portfolio': 'Mid-range to Premium',
        'rd_capabilities': 'Strong',
        'manufacturing': 'Contract Manufacturing + ODM',
        'distribution': ['Online', 'Retail Stores', 'Carrier Partnerships']
    }

    # Test different scenarios
    for i, event in enumerate(mobile_events, 1):
        print(f"\n🔍 SCENARIO {i}: {event['competitor']} - {event['event_type'].replace('_', ' ').title()}")
        print("-" * 60)

        strategic_context = f"""
        {event['competitor']} has made a significant move in the mobile market.
        This {'product launch' if event['event_type'] == 'product_launch' else event['event_type']}
        could impact our market position, especially in the {event.get('target_segment', 'general')} segment.
        We need to respond strategically to maintain our competitive position.
        """

        # Generate recommendations
        impact_score = 8.5 if event['competitor'].lower() == 'apple' else 7.8 if event['competitor'].lower() == 'samsung' else 6.5

        print(f"📊 Impact Score: {impact_score}/10")
        recommendations = agent.analyze_and_recommend(
            competitive_event=event,
            impact_score=impact_score,
            strategic_context=strategic_context,
            company_profile=mobile_company_profile
        )

        # Display key recommendations
        print(f"✅ Generated {len(recommendations)} mobile-specific recommendations:")

        for j, rec in enumerate(recommendations[:3], 1):  # Show top 3
            print(f"\n  {j}. {rec.title}")
            print(f"     📂 Category: {rec.category.value}")
            print(f"     ⚡ Priority: {rec.priority.value}")
            print(f"     ⏱️ Urgency: {rec.urgency_hours} hours")
            print(f"     🎯 Expected Impact: {rec.expected_impact}")
            print(f"     🔄 Mobile Context: {rec.rationale[:100]}...")

    print(f"\n" + "="*65)
    print("📱 SAMPLE MOBILE MARKET INTELLIGENCE REPORT")
    print("="*65)

    # Generate detailed report for Samsung scenario
    samsung_event = mobile_events[0]
    recommendations = agent.analyze_and_recommend(
        competitive_event=samsung_event,
        impact_score=8.5,
        strategic_context="Samsung Galaxy S25 Ultra represents major threat to premium segment",
        company_profile=mobile_company_profile
    )

    report = agent.generate_action_report(recommendations)
    print(report[:1500] + "...")  # Show first part of report

    # Show immediate actions for mobile market
    immediate_actions = agent.get_immediate_actions(recommendations)
    if immediate_actions:
        print(f"\n⚡ IMMEDIATE MOBILE ACTIONS REQUIRED:")
        print("-" * 40)
        for action in immediate_actions:
            print(f"🔥 {action.title}")
            print(f"   ⏰ Within {action.urgency_hours} hours")
            print(f"   📱 Focus: {action.category.value}")
            print()

    print("✅ Mobile Phone Market Action Recommender Agent demonstration complete!")

📱 Action Recommender Agent - Mobile Phone Competitive Intelligence

🔍 SCENARIO 1: Samsung - Product Launch
------------------------------------------------------------
📊 Impact Score: 7.8/10
✅ Generated 2 mobile-specific recommendations:

  1. Marketing Counter-Campaign
     📂 Category: Marketing & Communication
     ⚡ Priority: High
     ⏱️ Urgency: 120 hours
     🎯 Expected Impact: Medium
     🔄 Mobile Context: ...

  2. Feature Gap Analysis
     📂 Category: Product Strategy
     ⚡ Priority: High
     ⏱️ Urgency: 168 hours
     🎯 Expected Impact: Medium
     🔄 Mobile Context: ...

🔍 SCENARIO 2: Oppo - Pricing Change
------------------------------------------------------------
📊 Impact Score: 6.5/10
✅ Generated 1 mobile-specific recommendations:

  1. Mobile Customer Retention Program
     📂 Category: Marketing & Communication
     ⚡ Priority: Medium
     ⏱️ Urgency: 336 hours
     🎯 Expected Impact: Medium
     🔄 Mobile Context: Retention is more cost-effective than acquisition...

🔍